# Slender Blender
Débora Buzon da Silva

Dikson Ferreira dos Santos

Matheus Luis de Oliveira Silva

### Primeiro, importamos as bibliotecas necessárias.


In [233]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [234]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Aprenderemos GLSL conforme a necessidade do curso. Usaremos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal).
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso combinamos nossa variável vec2 com uma variável vec4. Além disso, modificamos nosso vetor com base nas transformações Model, View e Projection.

In [235]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal).
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, criaremos uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [236]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [237]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [238]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [239]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [240]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [241]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [242]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo da Aula, descrevemos o funcionamento desse formato.

In [243]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

In [244]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [245]:
vertices_list = []    
textures_coord_list = []

### Carregamos cada modelo e definimos funções para desenhá-los

In [246]:
modelo = load_model_from_file('../assets/table/wood_table.obj')
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cube.obj. Vertice final:',len(vertices_list))
table_start = 0
table_size = len(vertices_list) 

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0,'../assets/table/wood_table.png')

Processando modelo cube.obj. Vertice inicial: 0
Processando modelo cube.obj. Vertice final: 396


In [247]:
modelo = load_model_from_file('../assets/ground_stone/ground1.obj')
ground1_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
ground1_size = len(vertices_list) - ground1_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(1,'../assets/ground_stone/stone.jpg')

Processando modelo terreno.obj. Vertice inicial: 396
Processando modelo terreno.obj. Vertice final: 408


In [248]:
modelo = load_model_from_file('../assets/ground_forest/ground2.obj')
ground2_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
ground2_size = len(vertices_list) - ground2_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(4,'../assets/ground_forest/grass.jpg')

Processando modelo terreno.obj. Vertice inicial: 408
Processando modelo terreno.obj. Vertice final: 420


In [249]:
modelo = load_model_from_file('casa/casa.obj')
house_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
house_size = len(vertices_list) - house_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(2,'casa/casa.jpg')

Processando modelo casa.obj. Vertice inicial: 420
Processando modelo casa.obj. Vertice final: 1854


In [250]:
modelo = load_model_from_file('../assets/slenderman/slenderman.obj')
slender_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
slender_size = len(vertices_list) - slender_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3,'../assets/slenderman/slenderman.png')

Processando modelo monstro.obj. Vertice inicial: 1854
Processando modelo monstro.obj. Vertice final: 17838


In [251]:
## loading sky
modelo = load_model_from_file('../assets/sky/sky.obj')
sky_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
sky_size = len(vertices_list) - sky_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(5,'../assets/sky/animecloud.png')

Processando modelo monstro.obj. Vertice inicial: 17838
Processando modelo monstro.obj. Vertice final: 20718


In [252]:
# loading fence
modelo = load_model_from_file('../assets/fence/fence.obj')
fence_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
fence_size = len(vertices_list) - fence_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'../assets/fence/fence.jpg')

Processando modelo monstro.obj. Vertice inicial: 20718
Processando modelo monstro.obj. Vertice final: 22302


In [253]:
# loading chair
modelo = load_model_from_file('../assets/chair/chair.obj')
chair_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
chair_size = len(vertices_list) - chair_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(7,'../assets/chair/chair.png')

Processando modelo monstro.obj. Vertice inicial: 22302
Processando modelo monstro.obj. Vertice final: 23382


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Agora requisitamos dois slots.
* Um para enviar coordenadas dos vértices.
* Outro para enviar coordenadas de texturas.

In [ ]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

###  Enviando coordenadas de vértices para a GPU

In [ ]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [ ]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo.
* É necessário indicar qual o ID da textura do modelo.

In [ ]:
def draw_model(position, rotation, scale, start, model_size, texture_id):
    # aplica a matriz model
    
    # rotacao
    angle = rotation[0];
    r_x = rotation[1]; r_y = rotation[2]; r_z = rotation[3];
    
    # translacao
    t_x = position[0]; t_y = position[1]; t_z = position[2];
    
    # escala
    s_x = scale[0]; s_y = scale[1]; s_z = scale[2];
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, texture_id)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, start, model_size) ## renderizando

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional.
* Usei a posição do mouse para "direcionar" a câmera.

In [ ]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 0.5
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [ ]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    cameraPos[1] = 4;
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, exibimos a janela.

In [ ]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   

rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)   
    

    # draw sky  
    draw_model([0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.75, 0.75, 0.75], sky_start, sky_size, 5)
    # draw house
    draw_model([-50.0, 0.0, -30.0], [0.0, 0.0, 0.0, 1.0], [1.0, 1.0, 1.0], house_start, house_size, 2)
    # draw table
    draw_model([-50.0, 0.0, -30.0], [-90.0, 1.0, 0.0, 0.0], [0.25, 0.25, 0.25], table_start, table_size, 0)
    # draw four chairs around the table
    draw_model([-50.0, 3.0, -27.0], [0.0, 0.0, 0.0, 1.0], [0.35, 0.35, 0.35], chair_start, chair_size, 7)
    #draw_model([-50.0, 1.0, -30.0], [90.0, 0.0, 1.0, 0.0], [0.3, 0.3, 0.3], chair_start, chair_size, 7)
    #draw_model([-50.0, 1.0, -30.0], [180.0, 0.0, 1.0, 0.0], [0.3, 0.3, 0.3], chair_start, chair_size, 7)
    #draw_model([-50.0, 1.0, -30.0], [-90.0, 0.0, 1.0, 0.0], [0.3, 0.3, 0.3], chair_start, chair_size, 7)
    
    # draw both grounds
    draw_model([0.0, -1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [10.0, 10.0, 10.0], ground1_start, ground1_size, 1)
    draw_model([0.0, -1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [10.0, 10.0, 10.0], ground2_start, ground2_size, 4)
    
    # draw fence dividing spaces
    for i in range(-400, -10, 10):
        draw_model([0.0, -1.1, i], [90.0, 0.0, 1.0, 0.0], [30.0, 30.0, 30.0], fence_start, fence_size, 6)
    for i in range(0, 400, 10):
        draw_model([0.0, -1.1, i], [90.0, 0.0, 1.0, 0.0], [30.0, 30.0, 30.0], fence_start, fence_size, 6)

    # draw trees at random positions    

    # draws slenderman spinnig
    rotacao_inc += 2.0
    draw_model([30.0, -1., 30.0], [rotacao_inc, 0.0, 1.0, 0.0], [0.75, 0.75, 0.75], slender_start, slender_size, 3)
        
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    

    
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

* Adicione mais 2 modelos no cenário com suas respectivas texturas. Procure em repositórios abertos/gratuitos por modelos no formato Wavefront (extensão .obj). Verifique se o conteúdo das faces do modelo é baseado em triângulos. Verifique se o modelo acompanha alguma imagem (.jpg, png, etc) com a textura. Evite modelos compostos por múltiplos objetos/texturas.

* Coloque um cubo para "encapsular" todo o seu cenário. A face inferior do cubo será seu terreno. A face superior será o céu. As faces laterais serão horizontes. Crie um único arquivo de textura (imagem png ou jpg) com todas as faces. No arquivo .obj do modelo, define as coordenadas de textura para cada triângulo.